<a href="https://colab.research.google.com/github/asupraja3/ml-ng-notebooks/blob/main/Sigmoid_LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Optional Lab: Sigmoid Function & Logistic Regression

*Adapted, self-contained notebook inspired by Andrew Ng's Machine Learning Specialization.*

In this ungraded lab, you will:
- Explore the **sigmoid** (logistic) function.
- See how **logistic regression** uses the sigmoid to map a linear model into probabilities in \([0,1]\).



## Tools

We will use:
- **NumPy** for scientific computing.
- **Matplotlib** for plotting.
- **scikit-learn** for a simple logistic regression model.


In [ ]:

# === Setup ===
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# If running outside classic Jupyter, you might need: %matplotlib inline
np.set_printoptions(precision=4, suppress=True)



## Sigmoid (Logistic) Function

For classification, we want outputs between **0 and 1** (interpretable as probabilities).  
The **sigmoid** function maps any real-valued input \(z\) to \(g(z)\in(0,1)\):

\[
g(z) = \frac{1}{1 + e^{-z}}
\]

- As \(z \to -\infty\), \(g(z) \to 0\)  
- As \(z \to +\infty\), \(g(z) \to 1\)

Below, we implement it and try array/scalar inputs.


In [ ]:

# --- Exponential demo (like in-course walkthrough) ---

# Input is an array
input_array = np.array([1, 2, 3])
exp_array = np.exp(input_array)
print("Input to exp:", input_array)
print("Output of exp:", exp_array)

# Input is a single number (scalar)
input_val = 1.0
exp_val = np.exp(input_val)
print("Input to exp:", input_val)
print("Output of exp:", exp_val)


In [ ]:

# --- Sigmoid implementation ---
def sigmoid(z):
    """Vectorized sigmoid that works for scalars, 1D, or ND arrays."""
    # For better numerical stability on large |z|, use a piecewise form:
    z = np.array(z, dtype=float)
    out = np.empty_like(z, dtype=float)
    pos = z >= 0
    neg = ~pos
    out[pos] = 1.0 / (1.0 + np.exp(-z[pos]))
    # For negative z, rewrite: 1/(1+e^-z) = e^z / (1+e^z)
    ez = np.exp(z[neg])
    out[neg] = ez / (1.0 + ez)
    return out

# Try sigmoid with array & scalar
z_array = np.array([-6, -3, -1, 0, 1, 3, 6], dtype=float)
print("z array:", z_array)
print("sigmoid(z array):", sigmoid(z_array))

z_scalar = -2.0
print("z scalar:", z_scalar, " -> sigmoid(z):", float(sigmoid(z_scalar)))


In [ ]:

# --- Plot the sigmoid curve ---
z = np.linspace(-8, 8, 400)
g = sigmoid(z)

plt.figure(figsize=(6, 3))
plt.plot(z, g)
plt.axvline(0, linestyle='--', linewidth=1)
plt.axhline(0.5, linestyle='--', linewidth=1)
plt.xlabel("z")
plt.ylabel("g(z)")
plt.title("Sigmoid / Logistic Function")
plt.ylim(-0.05, 1.05)
plt.grid(True, alpha=0.3)
plt.show()



## Logistic Regression

A logistic regression model applies the sigmoid to the familiar linear model:

\[
f_{w,b}(x) = g(w^T x + b), \quad \text{where} \quad g(z)=\frac{1}{1+e^{-z}}.
\]

- The term \(z = w^T x + b\) is a **linear regression** output.  
- Passing \(z\) through **sigmoid** yields a **probability**: \(P(y=1\mid x)\).  
- The usual decision rule is: predict **1** if \(f_{w,b}(x) \ge 0.5\), else **0**.


In [ ]:

# --- Small 1D classification dataset (tumor size -> malignant?) ---
x_train = np.array([0., 1., 2., 3., 4., 5.])
y_train = np.array([0, 0, 0, 1, 1, 1])

# Reshape to (m, 1) for scikit-learn
X = x_train.reshape(-1, 1)
print("X shape:", X.shape, "y shape:", y_train.shape)


In [ ]:

# --- Fit logistic regression and inspect ---
logreg = LogisticRegression(solver='lbfgs')
logreg.fit(X, y_train)

# Probability for class 1
proba = logreg.predict_proba(X)[:, 1]
y_pred = (proba >= 0.5).astype(int)

print("w (coef):", logreg.coef_, "b (intercept):", logreg.intercept_)
print("Accuracy:", accuracy_score(y_train, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_train, y_pred))
print(classification_report(y_train, y_pred, digits=3))

# Plot points + probability curve
xs = np.linspace(x_train.min() - 0.5, x_train.max() + 0.5, 200).reshape(-1, 1)
probs = logreg.predict_proba(xs)[:, 1]

plt.figure(figsize=(6, 3))
plt.scatter(x_train[y_train==0], y_train[y_train==0], marker='o', label='y=0')
plt.scatter(x_train[y_train==1], y_train[y_train==1], marker='x', label='y=1')
plt.plot(xs, probs, label='P(y=1 | x)')
plt.axhline(0.5, linestyle='--', linewidth=1, label='0.5 threshold')
plt.xlabel("Tumor size (x)")
plt.ylabel("Probability / label")
plt.title("Logistic Regression on Categorical Data")
plt.ylim(-0.1, 1.1)
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()



## Notes / Try This

- Notice the orange line in the lecture corresponds to \(z = w^T x + b\), which is **not** a probability by itself.  
- The sigmoid wraps \(z\) into \([0,1]\), giving a probability curve.  
- These predictions match the pattern that larger tumors (right side) are more likely to be malignant (label 1).

**Exercises:**
1. Add additional data points near `x=9` or `x=10` (large size) and re-run the notebook. What happens to the curve and predictions?  
2. Change the decision threshold (e.g., 0.3 or 0.7) and observe the effect on predictions.  
3. Compare against a linear regression model with a 0.5 threshold and note the differences in behavior.
